In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
import re
import pandas as pd
from unidecode import unidecode
from tensorflow.keras.optimizers import Adam

### Carga de datasets

In [12]:
ruta = r"C:/Users/rhernandez/OneDrive - Generando Soluciones Anlalíticas S.A/Documents/New folder/Proyecto_Fase_2/datasets"
archivos = [os.path.join(ruta, f) for f in os.listdir(ruta) if f.endswith(".xlsx")]

dfs = {}

for archivo in archivos:
    base = os.path.basename(archivo)
    match = re.search(r"\d{4}", base)
    if match:
        anio = match.group()
        df = pd.read_excel(archivo)
        dfs[f"df_{anio}"] = df

for year in range(2020, 2024 + 1):
    name = f"df_{year}"
    if name in dfs:
        df = dfs[name]

        df.columns = [unidecode(c).lower() for c in df.columns]

        df.columns = [re.sub(r"gran_grupos|gran_grupos", "gran_grupos", c) for c in df.columns]
        df.columns = [re.sub(r"subg_primarios|subg_principales", "subg_principales", c) for c in df.columns]
        df.columns = [re.sub(r"g_primarios", "g_primarios", c) for c in df.columns]

        df = df.drop(columns=["edad_quinquenales", "ocupacionhabitual", "filter_$"], errors="ignore")

        if "area_geo_inf" in df.columns:
            df["area_geo_inf"] = df["area_geo_inf"].astype(str)

        dfs[name] = df

df_final = pd.concat([dfs[f"df_{y}"] for y in range(2020, 2024 + 1)], ignore_index=True)
df_final = df_final.drop(columns=["num_corre"], errors="ignore")
df_final["area_geo_inf"] = pd.to_numeric(df_final["area_geo_inf"], errors="coerce")
df_final.head()


,depto_boleta,muni_boleta,mes_boleta,ano_boleta,falta_inf,sexo_inf,edad_inf,grupo_etnico_inf,est_conyugal_inf,nacimiento_inf,...,niv_escolaridad_inf,est_ebriedad_inf,area_geo_inf,depto_nacimiento_inf,nacionalidad_inf,g_edad_80ymas,g_edad_60ymas,subg_principales,gran_grupos,g_primarios
0,1,102,1,2020,3,1,29,2,1,101,...,4,1,2,1,1,4,4,971,12,NaN
1,1,102,1,2020,3,1,19,2,1,101,...,3,1,2,1,1,2,2,94,9,NaN
2,1,102,1,2020,4,1,19,2,1,101,...,3,1,2,1,1,2,2,94,9,NaN
3,1,102,1,2020,3,1,22,2,1,101,...,3,1,2,1,1,3,3,31,3,NaN
4,1,102,1,2020,4,1,22,2,1,101,...,3,1,2,1,1,3,3,31,3,NaN


In [32]:
#df_final_bk = df_final.copy()
df_final = df_final_bk.copy()

In [33]:
df_final = df_final[df_final["area_geo_inf"] != 9].copy()
df_final["area_geo_inf"] = df_final["area_geo_inf"] - 1

In [34]:
df_final = df_final.sample(frac = 1, random_state=42).reset_index(drop = True)

In [35]:
X = df_final[['depto_boleta','sexo_inf', 'edad_inf', 'grupo_etnico_inf','est_conyugal_inf','cond_alfabetismo_inf', 'niv_escolaridad_inf','falta_inf','est_ebriedad_inf']]
y = df_final['area_geo_inf']

In [44]:
y.unique()

array([0, 1], dtype=int64)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
model = Sequential()
model.add(Dense(32, input_dim=9, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation ='sigmoid'))

c:\Users\rhernandez\Anaconda3\envs\agentes\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6455 - loss: 0.8313 - val_accuracy: 0.6591 - val_loss: 0.6342
Epoch 2/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6603 - loss: 0.6268 - val_accuracy: 0.6659 - val_loss: 0.6113
Epoch 3/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6652 - loss: 0.6192 - val_accuracy: 0.6691 - val_loss: 0.6085
Epoch 4/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6671 - loss: 0.6135 - val_accuracy: 0.6703 - val_loss: 0.6086
Epoch 5/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6706 - loss: 0.6064 - val_accuracy: 0.6750 - val_loss: 0.5998
Epoch 6/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6736 - loss: 0.6033 - val_accuracy: 0.6736 - val_loss: 0.6058
Epoch 7/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6729 - loss: 0.6015 - val_accuracy: 0.6749 - val_loss: 0.5973
Epoch 8/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6755 - loss: 0.5992 - 

In [23]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

836/836 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6439 - loss: -196397584.0000
Loss: -196397584.0, Accuracy: 0.6439416408538818


In [ ]:
personas_rd_1_1 = np.array([[1,30,2]])
res = model.predict(personas_rd_1_1)
print(res)